# EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# import httpimport
# url = 'https://raw.githubusercontent.com/zach-brown-18/class-toolkit/main/eda/'
# with httpimport.remote_repo(['cleaning'], url):
#     import cleaning as c

---

# Load and Look at Data

In [3]:
dice = pd.read_csv('../data/dice.csv', encoding='latin-1')
print(dice.shape)

(21919, 12)


In [4]:
# Drop the fluff
use_cols = ['date_added', 'job_description', 'job_title', 'job_type', 'location', 'organization', 'sector']
dice = dice.loc[:, use_cols]

In [5]:
# 7 total columns used
print(dice.shape)
dice.head(2)

(21919, 7)


,date_added,job_description,job_title,job_type,location,organization,sector
0,11/11/2016,"Minimum Required Skills:EDI, TrustedLink, AS2,...",EDI Analyst,"Full Time, Full-time, Employee","Stamford, CT",CyberCoders,"EDI, TrustedLink, AS2, VAN - EDI, TrustedLink,..."
1,11/11/2016,"InformaticaåÊ/ ETL DeveloperSt, Petersburg, FL...",Informatica ETL Developer,"Full Time, Full Time","St Petersburg, FL",TrustMinds,ETL Informatica B2B Data Exchange Netezza Orac...


---

# Cleaning

## Functions

In [331]:
def remove_special_chars(text):
    string_encode = text.encode("ascii", "ignore")
    return string_encode.decode()

In [332]:
def remove_duplicate_skills(idx,df):
    skills = df.loc[idx, 'sector'].split(' -')[0]
    try:
        df.loc[idx, 'job_description'] = df.loc[idx, 'job_description'].split(skills)[1]
    except:
        print(f'{idx} failed.')

In [333]:
def remove_phone_numbers(string):
    # regex pattern to match phone numbers of different forms
    regex1= "\w{3}-\w{3}-\w{4}"
    regex2= "(\w{3})\w{3}-\w{4}"
    
    # remove patterns
    replace_idx1 = [(m.start(0), m.end(0)) for m in re.finditer(regex1, string)]
    replace_idx2 = [(m.start(0), m.end(0)) for m in re.finditer(regex2, string)]
    patterns = [replace_idx1, replace_idx2]
    
    for pattern in patterns:
        if pattern:
            for start, stop in pattern:
                string = string[0: start:] + string[stop::]
    
    return string

In [334]:
def remove_emails(string):
    # regex pattern to match email addresses from different email providers
    regex1 = '\S+@\S+\.com'
    regex2 = '\S+@\S+\.net'
    
    # remove patterns
    replace_idx1 = [(m.start(0), m.end(0)) for m in re.finditer(regex1, string)]
    replace_idx2 = [(m.start(0), m.end(0)) for m in re.finditer(regex2, string)]
    patterns = [replace_idx1, replace_idx2]
    
    for pattern in patterns:
        if pattern:
            for start, stop in pattern:
                string = string[0: start:] + string[stop::]
    
    return string

In [335]:
def find_phone_numbers(string):
    regex1= "\w{3}-\w{3}-\w{4}"
    regex2= "(\w{3})\w{3}-\w{4}"
    
    if re.search(regex1, string) or re.search(regex2, string):
        return 1
    return 0

def find_emails(string):
    regex1 = '\S+@\S+\.com'
    regex2 = '\S+@\S+\.net'
    lst1, lst2 = re.findall(regex1, string), re.findall(regex2, string)
    
    if lst1 or lst2:
        return 1
    return 0

In [336]:
# function to get rid of repeated skills after a '-'
def split_skills(data):
        x = data
        nospace = x.replace(' ', '') # removing spaces
        if '-' in nospace:
            first_half = nospace.split('-')[0] #first half of string before '-'
            second_half = nospace.split('-')[1] # second half of string after '-'
            if first_half == second_half:
                x = list(x.split('-'))[0].strip() #getting only first half of string to return
                return x
            else:
                return x      
        else:
            return x

---

## Cleaning job_description

In [337]:
# Correct multiple spaces (including \t and \n)
dice['job_description'] = dice['job_description'].map(lambda x: re.sub('\s+',' ', x))
dice['sector'].fillna('', inplace=True)
dice['sector'] = dice['sector'].map(lambda x: re.sub('\s+',' ', x))

# Remove special characters
dice['job_description'] = dice['job_description'].map(remove_special_chars)
dice['sector'] = dice['sector'].map(remove_special_chars)

# Remove skills repeated in job description and sector
skills_repeated = dice['job_description'].map(lambda x: 'Minimum Required Skills' in x)
for row in dice.loc[skills_repeated, :].index:
    remove_duplicate_skills(row, dice)

11646 failed.
11655 failed.
11676 failed.
17688 failed.
17697 failed.
17733 failed.
17734 failed.
18800 failed.


## Search Duplicate Posts

In [338]:
# Find duplicate posts
description_counts = dice['job_description'].value_counts()
multiple_posts = description_counts > 1
description_counts[multiple_posts].sum()

3305

In [339]:
# CyberCoders are spamming the list
repeated_descriptions = description_counts[multiple_posts].index
repeated_descriptions = dice['job_description'].map(lambda x: x in repeated_descriptions)

dice.loc[repeated_descriptions, :]['organization'].value_counts().head()

CyberCoders               2706
TEKsystems                 103
Robert Half Technology      56
Citrix                      39
NORTHROP GRUMMAN            31
Name: organization, dtype: int64

In [340]:
# Majority of repeated posts are only repeated once
description_counts[description_counts == 2].sum()

1662

**Conclusion:** Repeated, identical posts will distort the data pool. If a post is repeated, drop all occurances but one.

## Drop Duplicate job posts

In [341]:
print(f'{dice.shape[0]} job listings before dropping duplicates')

21919 job listings before dropping duplicates


In [342]:
# Define duplicate posts as having identical job description, job type and organization.
non_duplicates = dice.drop(columns=['date_added', 'location', 'job_title', 'sector']).drop_duplicates().index
dice = dice.loc[non_duplicates, :].reset_index(drop=True)

print(f'{dice.shape[0]} job listings after dropping duplicates')

19919 job listings after dropping duplicates


In [343]:
# Find duplicate posts - still some left over
description_counts = dice['job_description'].value_counts()
multiple_posts = description_counts > 1
print(f'{description_counts[multiple_posts].sum()} duplicates left over')

37 duplicates left over


## Remove phone numbers and email addresses

In [344]:
# Track posts that have phone number or email address listed
# dice['has_email'] = dice['job_description'].map(find_emails)
# dice['has_phone_number'] = dice['job_description'].map(find_phone_numbers)

In [345]:
# Remove them from the job description
dice['job_description'] = dice['job_description'].map(remove_phone_numbers)
dice['job_description'] = dice['job_description'].map(remove_emails)

In [346]:
# Replace useless text with empty string
mask = dice['job_description'] == 'Please send resume with rate expectations.'
idx = dice[mask]['job_description'].index
dice.loc[idx, 'job_description'] = ''

In [347]:
dice.reset_index(drop=True, inplace=True)

In [348]:
dice[dice['job_description'] == '']

,date_added,job_description,job_title,job_type,location,organization,sector
1063,7/13/2016,,Software Engineer,"Full Time, Full Time","Greenwich, CT",U.S. Tech Solutions Inc.,"C++,ACCESS,Business Intelligence,Data Structur..."
5186,7/14/2016,,.Net Developer,"Full Time, Full Time","Waltham, MA",U.S. Tech Solutions Inc.,".Net,ACCESS,Amazon,Android,AWS,Business Intell..."
15825,5/9/2016,,Sr Java Application Developer (backend),"Full Time, 12+ months","Pasadena, CA",Acclaim Systems,"Java backend developer, Junit"
16633,6/10/2016,,Help Desk Analyst,"Contract W2, 06+ months","Hartford, CT",U.S. Tech Solutions Inc.,"Knowledge of Windows 7, Office 2013, and Ticke..."
17946,12/8/2016,,Sr Technical Project Manager (Human Services),"Contract W2, C2H Independent, 12+ months","Little Rock, AR",Acclaim Systems,"IT Project Management, Curam Cash Assistance, ..."
17997,12/8/2016,,Senior Curam Developer,"Full Time, Contract Corp-To-Corp, Contract Ind...","Little Rock, AR",Acclaim Systems,"IT Architecture, Curam Cash Assistance, Curam ..."


## Cleaning job_title

In [349]:
# Finding null values in job_title
dice[dice['job_title'].isnull()]

,date_added,job_description,job_title,job_type,location,organization,sector
13601,11/4/2016,Our end client is seeking technicians with exp...,NaN,NaN,NaN,NaN,


In [350]:
# Removing the one null job_title
dice = dice.dropna(subset = ['job_title'])

### catagorizing job_title

In [351]:
dice.loc[dice['job_title'].str.contains('Developer|developer|Dev|dev|Scrum|scrum'), 
                  'job_title'] = 'Developer'
dice.loc[dice['job_title'].str.contains('Analyst|analyst'), 
                  'job_title'] = 'Analyst'
dice.loc[dice['job_title'].str.contains('Programmer|programmer|Programming|programming|Full'),
                  'job_title'] = 'Programmer'
dice.loc[dice['job_title'].str.contains('Manager|manager|Project Coordinator|Technical Lead'), 
                  'job_title'] = 'Manager'
dice.loc[dice['job_title'].str.contains('Engineer|engineer'), 
                  'job_title'] = 'Engineer'
dice.loc[dice['job_title'].str.contains('Architect|architect'), 
                  'job_title'] = 'Architect'
dice.loc[dice['job_title'].str.contains('Designer|designer'), 
                  'job_title'] = 'Designer'
dice.loc[dice['job_title'].str.contains('Technician|technician'), 
                  'job_title'] = 'Technician'
dice.loc[dice['job_title'].str.contains('Administrator|administrator|Admin|admin'), 
                  'job_title'] = 'Administrator'
dice.loc[dice['job_title'].str.contains('Consulting|consulting|Consultation|consultation|Consultant|consultant'), 
                  'job_title'] = 'Consulting'
dice.loc[dice['job_title'].str.contains('Support|support|Helpdesk'), 
                  'job_title'] = 'Support'
dice.loc[dice['job_title'].str.contains('Director|director|CTO'),
                  'job_title'] = 'Director'
dice.loc[dice['job_title'].str.contains('Entry'), 
                  'job_title'] = 'Entry Position'
dice.loc[dice['job_title'].str.contains('Data Scientist|SQL DBA|SQL Server DBA|IT'), 
                  'job_title'] = 'Data Position'

In [352]:
dice['job_title'].value_counts()[0:13].sum()

16983

In [353]:
# Creating a list of the top 13 most popular job_titles
keep_titles = dice['job_title'].value_counts()[:13].index.tolist()
# Only keeping the top 13 job_titles from keep_titles
dice = dice[dice['job_title'].isin(keep_titles)]

In [354]:
dice['job_title'].value_counts()

Developer        5360
Engineer         4224
Analyst          1942
Manager          1417
Administrator     912
Architect         841
Consulting        598
Technician        353
Support           347
Programmer        325
Data Position     291
Designer          224
Director          149
Name: job_title, dtype: int64

## Cleaning location

### Catagorizing location

In [355]:
dice.loc[dice['location'].str.contains('ME|NH|VT|MA|MARYLAND|Laurel|CT|Ct|RI|NY|Ny|PA|NJ|Reading|OTHER|Portsmouth|West Chester'),
                  'location'] = 'Northeast United States'

dice.loc[dice['location'].str.contains('MI|OH|IN|IL|Il|WI|MN|IA|Ia|MO|Mo|KS|NE|ND|SD|Farmington|Lake County|Ashland'),
                  'location'] = 'Midwest United States'

dice.loc[dice['location'].str.contains('DE|MD|Md|DC|WV|VA|NC|SC|Anderson|KY|Ky|TN|GA|Ga|MS|AL|AR|OK|TX|Stafford|Austin|LA|La|FL|Boca Raton|South'),
                  'location'] = 'Southern United States'

dice.loc[dice['location'].str.contains('MT|WY|CO|Co|NM|AZ|UT|ID|Id|NV|CA|Ca|OR|WA|AK|HI|Hi|Pasadena|Redmond|Greenwood Village|San Francisco'),
                  'location'] = 'Western United States'

dice.loc[dice['location'].str.contains('Hyderabad|London|Bangalore|Dublin|Taguig City|ON|BC|Windsor'),
                  'location'] = 'International'

dice.loc[dice['location'].str.contains('STATE|City|Satellite Office; North America-us-il-chicago; Nor|United States Of America'),
                  'location'] = 'Unknown'
dice.head(3)

,date_added,job_description,job_title,job_type,location,organization,sector
0,11/11/2016,"If you are an EDI Analyst with experience, ple...",Analyst,"Full Time, Full-time, Employee",Northeast United States,CyberCoders,"EDI, TrustedLink, AS2, VAN - EDI, TrustedLink,..."
1,11/11/2016,"Informatica/ ETL DeveloperSt, Petersburg, FL O...",Developer,"Full Time, Full Time",Southern United States,TrustMinds,ETL Informatica B2B Data Exchange Netezza Orac...
2,11/11/2016,"Sunnyvale , CAANGULARJSdeveloperswithhibernat...",Developer,"Full Time, Contract Corp-To-Corp, Contract Ind...",Western United States,K Anand Corporation,Angular


In [356]:
# Creating a list of top 4 locations
keep_locations = dice['location'].value_counts()[:4].index.tolist()

# Only keeping the top 4 locations from list keep_locations
dice = dice[dice['location'].isin(keep_locations)]

In [357]:
dice['location'].value_counts()

Southern United States     5303
Western United States      4533
Northeast United States    4232
Midwest United States      2783
Name: location, dtype: int64

## Cleaning job_type

In [358]:
# Droping 238 rows of data that are null for job_type.
dice.dropna(subset = ['job_type'], inplace=True)
dice.reset_index(drop=True, inplace=True)

### Catagorizing job_type

In [359]:
dice.loc[dice['job_type'].str.contains('Full Time|Full-time|per year|per'),
                  'job_type'] = 'Full Time'

dice.loc[dice['job_type'].str.contains('Contract|contract|C2H'),
                  'job_type'] = 'Contract'

dice.loc[dice['job_type'].str.contains('Part Time'),
                  'job_type'] = 'Part Time'

dice.loc[dice['job_type'].str.contains('Market related|Market|Negotiable'),
                  'job_type'] = 'Market Dependent'

dice.loc[dice['job_type'].str.contains('-|EXPERIENCE|RELOCATION|define'),
                  'job_type'] = 'Unknown'

dice.head(3)

,date_added,job_description,job_title,job_type,location,organization,sector
0,11/11/2016,"If you are an EDI Analyst with experience, ple...",Analyst,Full Time,Northeast United States,CyberCoders,"EDI, TrustedLink, AS2, VAN - EDI, TrustedLink,..."
1,11/11/2016,"Informatica/ ETL DeveloperSt, Petersburg, FL O...",Developer,Full Time,Southern United States,TrustMinds,ETL Informatica B2B Data Exchange Netezza Orac...
2,11/11/2016,"Sunnyvale , CAANGULARJSdeveloperswithhibernat...",Developer,Full Time,Western United States,K Anand Corporation,Angular


In [360]:
dice['job_type'].value_counts()

Full Time           10389
Contract             6102
Market Dependent      126
Unknown                32
Part Time               9
Competitive             3
Name: job_type, dtype: int64

## Cleaning sector

In [361]:
# renaming sector to be more descriptive
dice=dice.rename(columns = {'sector':'skills'})

In [362]:
# lowercasing skills
dice['skills']= dice['skills'].str.lower()

### Removing commas, backslashes and extra spaces

In [363]:
# replacing commas with a space
dice['skills'] = dice['skills'].str.replace(',', ' ')

In [364]:
# replcaing / with space
dice['skills'] = dice['skills'].str.replace('/', ' ')

In [365]:
# making sure no extra spaces. help from : https://stackoverflow.com/questions/43071415/remove-multiple-blanks-in-dataframe
dice['skills'] = dice['skills'].replace('\s+', ' ', regex=True)

### Looking at skills string length

In [366]:
# creating a new column for skills string length
dice['skills_len'] = dice['skills'].str.split().map(lambda x: len(x))

In [367]:
dice[dice['skills_len'] ==2].head(3)

,date_added,job_description,job_title,job_type,location,organization,skills,skills_len
18,11/12/2016,Senior Java DeveloperLocation: SunnyvaleStart ...,Developer,Contract,Western United States,TEKsystems,java developer,2
30,11/11/2016,"Client is looking for a ""AEM CQ5 Developer"" in...",Developer,Contract,Northeast United States,Nutech Information Systems,aem cq5,2
31,11/12/2016,As the Project Manager for and HR Solutions te...,Manager,Contract,Western United States,TEKsystems,project manager,2


In [368]:
dice[dice['skills_len'] ==3].head(3)

,date_added,job_description,job_title,job_type,location,organization,skills,skills_len
6,11/11/2016,Linux System Administrator opportunity- with i...,Administrator,Full Time,Northeast United States,Landover Assocates,linux system administrator,3
34,11/11/2016,The suitable candidate should be familier with...,Architect,Full Time,Southern United States,Sanrose Information Services Inc.,apex data solutions,3
39,11/12/2016,**Must have at least 1 year of professional Na...,Designer,Contract,Western United States,TEKsystems,mobile ux designer,3


We can see that many of the skills are actually just the title of the job or job type. These should be changed to an empty string.

### Cleaning skills of already described features

In [369]:
# if skill is just the job title replace with a empty string
dice.loc[(dice['skills'] ==  dice['job_title'].str.lower()), 'skills'] = ""

In [370]:
dice['skills'].value_counts().head(10)

see job description                162
contract w2                         71
full time                           56
                                    38
network engineer                    30
please refer to job description     25
project manager                     22
.net developer                      22
business analyst                    20
c2h w2                              18
Name: skills, dtype: int64

In [371]:
# changing additional non-skills to empty strings
dice.loc[(dice['skills'] == 'Telecommuting not available Travel not required'.lower()), 'skills'] = ''
dice.loc[(dice['skills'] == 'Full Time'.lower()), 'skills'] = ''
dice.loc[(dice['skills'] == 'Contract W2'.lower()), 'skills'] = ''
dice.loc[(dice['skills'] == 'tad pgs inc. specializes in delivering secure reliable and rapidly implemented workforce solutions to the u.s. federal marketplace including u.s. government agencies and their prime contractors. wi'), 'skills'] = ''
dice.loc[(dice['skills'] == 'see job description'), 'skills'] = ''
dice.loc[(dice['skills'] == 'please refer to job description'), 'skills'] = ''
dice.loc[(dice['skills'] == '(see job description)'), 'skills'] = ''
dice.loc[(dice['skills'] == 'refer to job description'), 'skills'] = ''

### Dealing with duplicate words

In [372]:
# removing duplicate words in a row
# help from:https://stackoverflow.com/questions/47316783/python-dataframe-remove-duplicate-words-in-the-same-cell-within-a-column-in-pyt
dice['skills'] = dice['skills'].str.replace(r'\b(\w+)(\s+\1)+\b', r'\1')

In [373]:
# some postings had repeat word after -
# mapping split_skills to whole skills row
dice['skills'] = dice['skills'].map(split_skills)

# Final DataFrame

In [374]:
dice.reset_index(drop=True, inplace=True)

In [375]:
dice

,date_added,job_description,job_title,job_type,location,organization,skills,skills_len
0,11/11/2016,"If you are an EDI Analyst with experience, ple...",Analyst,Full Time,Northeast United States,CyberCoders,edi trustedlink as2 van,9
1,11/11/2016,"Informatica/ ETL DeveloperSt, Petersburg, FL O...",Developer,Full Time,Southern United States,TrustMinds,etl informatica b2b data exchange netezza orac...,8
2,11/11/2016,"Sunnyvale , CAANGULARJSdeveloperswithhibernat...",Developer,Full Time,Western United States,K Anand Corporation,angular,1
3,11/12/2016,This nationally recognized Microsoft Gold Part...,Manager,Full Time,Western United States,Nigel Frank International,microsoft dynamics ax project manager - toront...,10
4,11/11/2016,"If you are a .NET Developer with experience, p...",Developer,Full Time,Northeast United States,CyberCoders,c# asp.net sql javascript mvc,11
...,...,...,...,...,...,...,...,...
16656,12/9/2016,Job Description JPMorgan Chase & Co. (NYSE: J...,Developer,Full Time,Northeast United States,JPMorgan Chase,.net architecture developer development git ht...,26
16657,12/9/2016,Seeking Jr. Systems Administrators with experi...,Administrator,Contract,Midwest United States,TEKsystems,jr. linux administrator,3
16658,12/9/2016,If you are a Senior/Lead Devops Engineer with ...,Developer,Full Time,Midwest United States,CyberCoders,amazon web services linux bash ruby python agile,17
16659,12/9/2016,"Headquartered in downtown San Francisco, CA we...",Developer,Full Time,Western United States,CyberCoders,javascript react.js golang startup ror iot ana...,23


In [376]:
# exporting final dataframe to csv
dice.to_csv('../data/job_postings.csv', index=False)